In [2]:
import pandas as pd
import numpy as np
from cmdstanpy import CmdStanModel
import os
import arviz as az

In [3]:
y_data = pd.read_csv(os.path.realpath("../data/y_count_pwr.csv"), usecols=["y"])
age_data = pd.read_csv(os.path.realpath("../data/x_age.csv"), usecols=["age"])
ship_data = pd.read_csv(os.path.realpath("../data/ship_index.csv"), usecols=["ship"])
ship_engine_mapping_data = pd.read_csv(os.path.realpath("../data/engine_index.csv"), usecols=["engine"])
engine_data = pd.DataFrame({"engine": np.zeros(y_data.shape[0], dtype=np.int32)})
for x in range(y_data.shape[0]):
    engine_data.at[x, "engine"] = ship_engine_mapping_data.at[ship_data.at[x, "ship"]-1, "engine"]


In [4]:
engine_count = np.max(engine_data["engine"])
ship_count = np.max(ship_data["ship"])
max_age = np.max(age_data["age"])
data_count = y_data.shape[0]
engine_count, ship_count, max_age, data_count

(5, 99, 31, 653)

In [4]:
gp_model_dir = "models/hier_gp_weak/hier_gp_weak.stan"
gp_model = CmdStanModel(stan_file=gp_model_dir) #compile_model(layer3_path)

INFO:cmdstanpy:compiling stan program, exe file: /home/dashadower/git_repos/aria/regression/failure_bma/gaussian_process/stancode/hier_gp_weak
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /home/dashadower/git_repos/aria/regression/failure_bma/gaussian_process/stancode/hier_gp_weak


In [5]:
data = {
    "N": int(data_count),
    "N_engines": int(engine_count),
    "N_ships": int(ship_count),
    "N_ages_obs": int(max_age),
    "N_ages": int(max_age),
    "ship_engine_ind": ship_engine_mapping_data["engine"].values.tolist(),
    "ship_ind": ship_data["ship"].values.tolist(),
    "age_ind": age_data["age"].values.tolist(),
    "y": y_data["y"].values.tolist(),
}

In [6]:
%%time
gp_model_fit = gp_model.sample(chains=4, cores=4, data=data)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 1


CPU times: user 2.01 s, sys: 88.9 ms, total: 2.1 s
Wall time: 1min 40s


In [7]:
gp_model_fit.diagnose()

INFO:cmdstanpy:Processing csv files: /tmp/tmplku7l3zp/hier_gp_weak-202008102121-1-__a9u0zo.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-2-pyvdwgro.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-3-pipo39uy.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-4-26l23nsr.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
34 of 4000 (0.85%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete.


"Processing csv files: /tmp/tmplku7l3zp/hier_gp_weak-202008102121-1-__a9u0zo.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-2-pyvdwgro.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-3-pipo39uy.csv, /tmp/tmplku7l3zp/hier_gp_weak-202008102121-4-26l23nsr.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\n34 of 4000 (0.85%) transitions ended with a divergence.\nThese divergent transitions indicate that HMC is not fully able to explore the posterior distribution.\nTry increasing adapt delta closer to 1.\nIf this doesn't remove all divergences, try to reparameterize the model.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete."

In [13]:
pd.set_option('display.max_rows', None)
#gp_model_fit.summary()

In [14]:
pd.reset_option("display.max_rows")

In [10]:
y_new_pred = gp_model_fit.get_drawset(params = ["y_new_pred"])

In [11]:
az_inference = az.from_cmdstanpy(gp_model_fit, posterior_predictive="y_new_pred", log_likelihood="loglik", observed_data={"y": y_data["y"].values.tolist()})

In [12]:
az_inference

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> observed_data